In [ ]:
# import importlib

# # 1. 변경된 모듈을 평소처럼 import 합니다. (예: 'my_module.py'를 import)
# import my_module 

# # 2. 모듈의 내용이 변경되었다면, 아래 코드를 실행하여 다시 로드합니다.
# importlib.reload(my_module)

In [6]:
from db_handler import get_connection

In [ ]:
conn = get_connection